In [8]:
import subprocess
import numpy as np
import pandas as pd

In [9]:
metadata = pd.read_csv("../allele_specific_expression.csv")

In [12]:
metadata = metadata[metadata["DARPA ID"].notna()].reset_index(drop=True)

In [13]:
metadata

,GS_ID,CRF ID,DARPA ID,Has5mC,SeqType
0,FL22Z_1001,Apal-002,A22086,0.0,Illumina
1,FL22Z_1002,Apal-008,A22165,0.0,Illumina
2,FL22Z_1003,Apal-019,20148,1.0,HiFi
3,FL22Z_1005,Apal-076,A22149,0.0,Illumina
4,FL22Z_1006,Apal-030,20151,1.0,HiFi
5,FL22Z_1008,Apal-015,20153,1.0,HiFi
6,FL22Z_1009,Apal-028,20154,1.0,HiFi
7,FL22Z_1010,Apal-063,20155,1.0,HiFi
8,FL22Z_1011,Apal-011,20156,1.0,HiFi
9,FL22Z_1012,Apal-069,A22084,0.0,Illumina


In [14]:
!ls -lh /storage/group/ibb3/default/global_search/genotyping_samples_symlinks/

total 0
lrwxrwxrwx 1 nsl5160 ibb3_collab 124 May 10 15:11 CU22Z_1001_RE_1.fq.gz -> /storage/group/ibb3/default/global_search_Tf/BMK230125-BG405-ZX01-0101-190samples-Data/rawdata/Unknown_BG405-02T0001_1.fq.gz
lrwxrwxrwx 1 nsl5160 ibb3_collab 124 May 10 15:11 CU22Z_1001_RE_2.fq.gz -> /storage/group/ibb3/default/global_search_Tf/BMK230125-BG405-ZX01-0101-190samples-Data/rawdata/Unknown_BG405-02T0001_2.fq.gz
lrwxrwxrwx 1 nsl5160 ibb3_collab 124 May 10 15:11 CU22Z_1002_1.fq.gz -> /storage/group/ibb3/default/global_search_Tf/BMK230125-BG405-ZX01-0101-190samples-Data/rawdata/Unknown_BG405-02T0002_1.fq.gz
lrwxrwxrwx 1 nsl5160 ibb3_collab 124 May 10 15:11 CU22Z_1002_2.fq.gz -> /storage/group/ibb3/default/global_search_Tf/BMK230125-BG405-ZX01-0101-190samples-Data/rawdata/Unknown_BG405-02T0002_2.fq.gz
lrwxrwxrwx 1 nsl5160 ibb3_collab 124 May 10 15:11 CU22Z_1003_1.fq.gz -> /storage/group/ibb3/default/global_search_Tf/BMK230125-BG405-ZX01-0101-190samples-Data/rawdata/Unknown_BG405-02T0003_1.fq.gz
l

In [20]:
for i in metadata["GS_ID"].tolist():
    task = "trimgalore"
    mem = "24"
    cpus = "8"
    inpath = "/storage/group/ibb3/default/global_search/genotyping_samples_symlinks/"
    r1 = "*_1.fq.gz"
    r2 = "*_2.fq.gz"
    prefix = i
    outpath = "/storage/home/nsl5160/scratch/darpa/trimmed/"
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + prefix + r1 + " " + 
         inpath + prefix + r2 + " " + 
         outpath + " " + prefix], shell=True)

Submitted batch job 5254571


sbatch: slurm_job_submit: No account set, using default acct: open
sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 5254572
Submitted batch job 5254573


sbatch: slurm_job_submit: No account set, using default acct: open
sbatch: slurm_job_submit: No account set, using default acct: open
sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 5254574
Submitted batch job 5254575
Submitted batch job 5254576
Submitted batch job 5254577


sbatch: slurm_job_submit: No account set, using default acct: open
sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 5254578
Submitted batch job 5254579


sbatch: slurm_job_submit: No account set, using default acct: open
sbatch: slurm_job_submit: No account set, using default acct: open


Submitted batch job 5254580


sbatch: slurm_job_submit: No account set, using default acct: open


In [21]:
task = "star_index"
mem = "200"
cpus = "20"
subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
     cpus + " ../" + task + ".sh "], shell=True)

Submitted batch job 5254585


CompletedProcess(args=['sbatch --mem=200g --ntasks=20 ../star_index.sh '], returncode=0)

In [30]:
for i in range(len(metadata)):
    task = "star_map"
    mem = "128"
    cpus = "20"
    trimpath = "/storage/home/nsl5160/scratch/darpa/trimmed/"
    outpath = "/storage/home/nsl5160/scratch/darpa/mapped/"
    prefix = str(metadata["GS_ID"][i])
    vcfpath = "/storage/home/nsl5160/scratch/darpa/whatshap/" + str(metadata["DARPA ID"][i])
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         trimpath + " " + outpath + " " + prefix + " " + vcfpath], shell=True)

Submitted batch job 5254780
Submitted batch job 5254781
Submitted batch job 5254782
Submitted batch job 5254783
Submitted batch job 5254784
Submitted batch job 5254785
Submitted batch job 5254786
Submitted batch job 5254787
Submitted batch job 5254788
Submitted batch job 5254789


In [ ]:
conda activate phaser
cd /storage/group/ibb3/default/global_search/allelic_imbalance/

python phaser/phaser/phaser.py \
    --vcf ./calls/14120_depth_gq_biallelic_filtered_phased.vcf.gz \
    --bam ./mapped/RR164.wasp_filtered.bam \
    --paired_end 1 \
    --mapq 255 \
    --baseq 10 \
    --sample 14120 \
    --threads 20 \
    --id_separator "@" \
    --o RR164 \
    --pass_only 0

python phaser/phaser_gene_ae/phaser_gene_ae.py \
    --haplotypic_counts RR164.haplotypic_counts.txt \
    --features ./references/Acropora_cervicornis.gene.bed \
    --id_separator "@" \
    --o RR164_gene_ae.txt